# Baseline Models

We will develop the baseline neural network model (one hidden layer, 3-node softmax output) for comparisons for later architectures.

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# Reading in Data
data_path = 'data/data_upsampled.csv'
data_final = pd.read_csv(data_path)

In [5]:
# LOPO Splitting
def train_test_split(protein, curr_data):
    train_data = curr_data[data.protein != protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    test_data = curr_data[data.protein == protein].drop(['protein', 'pdb', 'resnum'], axis=1)
    
    y_train = train_data.type
    x_train = train_data.drop(['type'], axis=1)
    y_test = test_data.type
    x_test = test_data.drop(['type'], axis=1)

    return x_train, y_train, x_test, y_test

In [7]:
# Baseline Model
def baseline_model():
    model = Sequential()
    model.add(Dense(800, input_dim = 968, activation = 'relu'))
    model.add(Dense(3, activation='softmax'))
    opt = optimizers.Adam(learning_rate = 0.1)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return(model)

In [8]:
# Baseline Analysis
def baseline(protein):
    # Split Training Data based on Protein
    x_train, y_train, x_test, y_test = train_test_split(protein, data_final)
    
    # Build Model
    # 40 epochs based on initial runs and graphing of loss over time.
    estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=10, verbose=1)
    estimator.fit(x_train, y_train, verbose=1)
    
    # Generate Analysis Characteristics
    probs = estimator.predict_proba(x_test)
    predictions = estimator.predict(x_test)
    precision = precision_score(y_test, predictions, average="macro", zero_division=0)
    score = estimator.score(x_test, y_test)

    print('Accuracy: {}'.format(score))
    print('Precision: {}'.format(precision))
    
    # Plotting Confusion Matrices
    cmatrix = confusion_matrix(y_test, predictions)
    fig, ax = plt.subplots()
    sns.heatmap(cmatrix, xticklabels = ['Beneficial', 'Deleterious', 'Neutral'], yticklabels = ['Beneficial', 'Deleterious', 'Neutral'])
    ax.set_title('{}'.format(protein))
    plt.savefig('img/baseline-cmatrix/' + protein + '-baseline-cmatrix.png', dpi=300)

In [9]:
proteins = list(data_final['protein'].unique())
for protein in proteins:
    baseline(protein)

Epoch 1/20
24441/24441 [==============================] - 10s 401us/step - loss: 126.4940 - accuracy: 0.6574
Epoch 2/20
24441/24441 [==============================] - 9s 388us/step - loss: 0.8821 - accuracy: 0.6613
Epoch 3/20
24441/24441 [==============================] - 10s 389us/step - loss: 0.8817 - accuracy: 0.6613
Epoch 4/20
24441/24441 [==============================] - 9s 389us/step - loss: 0.8831 - accuracy: 0.6613
Epoch 5/20
24441/24441 [==============================] - 10s 393us/step - loss: 0.8842 - accuracy: 0.6613
Epoch 6/20
 3040/24441 [==>...........................] - ETA: 8s - loss: 0.8785 - accuracy: 0.6635

KeyboardInterrupt: 